# RE:从零开始的深度学习计算

In [2]:
import torch
from torch import nn
from torch.nn import functional as f

## 块

### 自定义块

In [5]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.out = nn.Linear(256,10)

    def forward(self,X):
        return self.out(f.relu(self.hidden(X)))

In [6]:
net = MLP()
X = torch.rand(2,20)
net(X)

tensor([[ 0.1115,  0.1298, -0.1694, -0.0586,  0.0038,  0.1882, -0.0331, -0.0936,
          0.1540,  0.0185],
        [ 0.2354,  0.3212, -0.1794,  0.0747,  0.0052,  0.2095, -0.1135, -0.1653,
          0.1494, -0.0699]], grad_fn=<AddmmBackward0>)

### 顺序块

In [9]:
class MySequential(nn.Module):
    def __init__(self,*args):
        super().__init__()
        for idx,module in enumerate(args):
            self._modules[str(idx)] = module
    def forward(self,X):
        for block in self._modules.values():
            X = block(X)
        return X


In [10]:
net = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

tensor([[ 0.0608, -0.0063, -0.0394,  0.2684,  0.1132, -0.2872, -0.0125,  0.1606,
         -0.1388,  0.1312],
        [ 0.0048,  0.0534,  0.0328,  0.2981,  0.1204, -0.2036, -0.2625,  0.1198,
         -0.3275,  0.0394]], grad_fn=<AddmmBackward0>)

作用在于把其他模块串起来

### 在前向传播函数中执行代码

In [14]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20,20),requires_grad=False)
        self.linear = nn.Linear(20,20)

    def forward(self,X):
        X = self.linear(X)
        X = f.relu(torch.mm(X,self.rand_weight)+1)
        X = self.linear(X)
        while X.abs().sum() > 1 :
            X /=2
        return X.sum()

In [15]:
net = FixedHiddenMLP()
net(X)

tensor(-0.2146, grad_fn=<SumBackward0>)

### 嵌套块

In [16]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20,64),nn.ReLU(),nn.Linear(64,32),nn.ReLU())
        self.linear = nn.Linear(32,16)

    def forward(self,X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(),nn.Linear(16,20),FixedHiddenMLP())
chimera(X)

tensor(-0.0606, grad_fn=<SumBackward0>)

## 参数管理

In [3]:
net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
X = torch.rand(size=(2,4))
net(X)

tensor([[ 0.1143],
        [-0.0526]], grad_fn=<AddmmBackward0>)

### 参数访问

In [4]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.3456,  0.1517,  0.3472,  0.3193, -0.1719,  0.0084,  0.2499, -0.1458]])), ('bias', tensor([-0.2098]))])


In [5]:
print(type(net[2].bias))

<class 'torch.nn.parameter.Parameter'>


In [6]:
print(net[2].bias)

Parameter containing:
tensor([-0.2098], requires_grad=True)


In [7]:
print(net[2].bias.data)

tensor([-0.2098])


In [8]:
print(net[2].bias.grad)

None


#### 访问全部参数

In [9]:
print(*[(name,param.shape) for name,param in net[0].named_parameters()])
print(*[(name,param.shape) for name,param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


#### 从嵌套块收集参数

In [10]:
def block1():
    return nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,4),nn.ReLU())
def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}',block1())
    return net

rgnet = nn.Sequential(block2(),nn.Linear(4,1))
rgnet(X)

tensor([[-0.0627],
        [-0.0627]], grad_fn=<AddmmBackward0>)

In [11]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [12]:
rgnet[0][1][0].bias.data

tensor([ 0.4502, -0.0411,  0.4406, -0.1110, -0.3651, -0.0281, -0.2059,  0.3768])

### 参数初始化

#### 内置初始化

In [13]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight,mean=0,std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([ 0.0033, -0.0220, -0.0039,  0.0005]), tensor(0.))

In [14]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight,1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [16]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight,42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.6014, -0.5899,  0.4587, -0.1929])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


#### 自定义初始化

$$
w \sim 
\begin{cases}
U(5,10), & \text{概率 } \frac{1}{4} \\[4pt]
0, & \text{概率 } \frac{1}{2} \\[4pt]
U(-10,-5), & \text{概率 } \frac{1}{4}
\end{cases}
$$

In [17]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init",*[(name,param.shape)
                      for name,param in m.named_parameters()][0])
        nn.init.uniform_(m.weight,-10,10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]
        

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 8.0714,  0.0000,  5.2386,  6.7874],
        [-8.1168,  8.3596,  8.4095,  5.2292]], grad_fn=<SliceBackward0>)

In [18]:
net[0].weight.data[:]+=1
net[0].weight.data[0,0]=42
net[0].weight.data[0]

tensor([42.0000,  1.0000,  6.2386,  7.7874])

#### 参数绑定

In [19]:
shared = nn.Linear(8,8)
net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),
                   shared,nn.ReLU(),
                   shared,nn.ReLU(),
                   nn.Linear(8,1))
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0,0]=100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


### 延后初始化

## 层

### 无参层

In [20]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self,X):
        return X - X.mean()

In [21]:
layer = CenteredLayer()
layer(torch.FloatTensor([1,2,3,4,5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [22]:
net = nn.Sequential(nn.Linear(8,128),CenteredLayer())

In [24]:
Y = net(torch.rand(4,8))
Y.mean()

tensor(9.3132e-09, grad_fn=<MeanBackward0>)

### 带参层

In [29]:
class MyLinear(nn.Module):
    def __init__(self,in_units,units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units,units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self,X):
        linear = torch.matmul(X,self.weight.data) + self.bias.data
        return f.relu(linear)

In [30]:
linear = MyLinear(5,3)
linear.weight

Parameter containing:
tensor([[ 0.0679, -0.0529,  0.9204],
        [-1.7568,  0.8764,  0.0929],
        [-0.0077,  1.0132, -0.3616],
        [ 0.2187, -2.4579, -0.1960],
        [ 1.1376,  1.0557, -1.6061]], requires_grad=True)

In [31]:
linear(torch.rand(2,5))

tensor([[0.9611, 1.4377, 0.9551],
        [0.8497, 0.0000, 1.4228]])

In [32]:
net = nn.Sequential(MyLinear(64,8),MyLinear(8,1))
net(torch.rand(2,64))

tensor([[0.0000],
        [1.6406]])

## 读写文件

In [33]:
import torch
from torch import nn
from torch.nn import functional as f

x=torch.arange(4)
torch.save(x,'x-file')

### 加载和保存张量

In [34]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [35]:
y = torch.zeros(4)
torch.save([x,y],'x-file')

In [36]:
x2,y2 = torch.load('x-file')
(x2,y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [37]:
mydict ={'x':x,'y':y}
torch.save(mydict,'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

### 加载和保存模型参数

In [38]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.output = nn.Linear(256,10)

    def forward(self,x):
        return self.output(f.relu(self.hidden(x)))

In [39]:
net =MLP()
X = torch.randn(size=(2,20))
Y = net(X)

In [42]:
torch.save(net.state_dict(),'mlp.params')

In [43]:
clone =MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [1]:
!nvidia-smi

Sat Feb 14 14:52:13 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 572.60                 Driver Version: 572.60         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   71C    P5              6W /   80W |    2735MiB /   8188MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
import torch
print(f"PyTorch 版本: {torch.__version__}")
print(f"CUDA 是否可用: {torch.cuda.is_available()}")
print(f"CUDA 版本: {torch.version.cuda}")
print(f"GPU 名称: {torch.cuda.get_device_name(0)}")

PyTorch 版本: 1.12.0+cu116
CUDA 是否可用: True
CUDA 版本: 11.6
GPU 名称: NVIDIA GeForce RTX 4060 Laptop GPU
